<a href="https://colab.research.google.com/github/JaimeRedondo/Mushrooms_Classification/blob/master/Mushrooms_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#We can search images from Bing, better option because it works.
! pip install bing-image-downloader
! pip install opencv-python
#instalo open cv
from matplotlib import image
from matplotlib import pyplot
from skimage import io
import cv2
import os
! pip install tensorflow

import tensorflow as tf
#Packages
#--Web scraping packages
from bs4 import BeautifulSoup
import requests
#Pandas/numpy for data manipulation
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bing_image_downloader import downloader
from tqdm.notebook import tqdm
from time import sleep 

#import libraries I want.
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import Input

%pylab inline
plt.style.use('seaborn-talk')

Populating the interactive namespace from numpy and matplotlib


# Define Functions

Web Scraping to have enough mushrooms types.

In [2]:
def scraping_mushrooms():    
    clean_titles_all=pd.DataFrame()
    titles=['Final']
    url = 'https://www.fungipedia.org/hongos.html?start=0'

    while 'Final' in list(titles):
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
    
    ## getting the web titles
        titles=[]
        all_links = soup.find_all("a")
        for link in all_links:
            titles.append(link.get("title"))
        
        titles=pd.Series(titles).apply(str)
    
        titles=titles[titles!='None']
    
        titles_df=titles.reset_index()
    
        for phrase in titles_df[0]:
            if phrase =='Catálogo de Setas y Hongos':
                numero=int((titles_df[titles_df[0]==phrase]['index']))
            else:
                continue
            
        titles=titles[titles.index > numero]
    
        clean_titles=titles[titles.str.len()>6][:-1]
    
        clean_titles=clean_titles.to_frame()
    
        siguiente = '='+str(int(url.split('=')[1])+20)
    
        url = url.split('=')[0] + siguiente
    
        clean_titles_all = clean_titles_all.append(clean_titles)
    
    clean_titles_all=clean_titles_all.reset_index()

    del clean_titles_all['index']

    lista_setas = clean_titles_all[0]

    lista_setas=list(lista_setas)
    
    return lista_setas


Download Images From an Internet Browser (Bing)

In [3]:
#keep 1 photo of ecah mushroom,##write a list of things in the input of searcher:
def searcher_downloader(lista):
    for seta in tqdm(lista):
        try:
            downloader.download(seta, limit=20,  output_dir='dataset', 
            adult_filter_off=True, force_replace=False, timeout=60)
        except:
            print("esta no puedo")
    print("I'm done")

Standarize the images downloaded

In [4]:
#estandarizamos nombres de fotos
def standarize_names(directory):
    for folder in tqdm(directory):
        directorio = os.listdir('dataset/'+folder)
        for image in directorio:     
            if image.split('.')[1] == 'jpg':
                continue
            else:
                image_new=image.split('.')[0] + '.jpg'
                os.rename('dataset/'+folder+'/'+image, 'dataset/'+folder+'/'+image_new)

In [5]:
#resize all images shape in 200x200 and print what photos are not available
def resize_photos(directory):
    for folder in tqdm(directory):
        for numero in range(20):
            try:
                route = "dataset/" + folder +'/Image_'+ str(numero+1) + '.jpg'
                data_picture = io.imread(route)
                data_picture = cv2.resize(data_picture, (200, 200))
                # save the images
                cv2.imwrite(route,data_picture)
            except:  
                print("imagen "+str(numero+1)+' de la seta '+folder+' no está disponible')

In [6]:
#standarize 0_255 pixels to 0 - 1.
lista=[]
def standarize_pixels(directory):
    for folder in tqdm(directory):
        for numero in range(20):
            try:
                route = "dataset/" + folder +'/Image_'+ str(numero+1) + '.jpg'
                image_rgb=io.imread(route)/255.0
                lista.append(image_rgb)
            except:
                print(folder+str(numero+1))

In [7]:
def standarize_photos(directory):
    print("renombrando...")
    standarize_names(directory)
    print("homogeneizando tamaño fotos...")
    resize_photos(directory)
    print("estandarizando pixels...")
    standarize_pixels(directory)

In [8]:
def comprobar_longitudes_carpetas(directorio):
    for folder in directorio:
        print(folder + ' '+str(len(os.listdir("dataset/" + folder))))

In [9]:
def plot_mushroom_image(N, imgs, labels):
    print(labels[N])
    plt.imshow(imgs[N,], cmap=plt.cm.binary)
    plt.grid(True)

### Apply the def functions

In [10]:
lista_setas=scraping_mushrooms()

In [11]:
lista_setas_2=lista_setas[0:10]
#primero lo hare con unm muestra

In [12]:
lista_setas_2

['Abortiporus biennis',
 'Adelphella babingtonii',
 'Agaricus arvensis',
 'Agaricus campestris',
 'Agaricus litoralis',
 'Agaricus sylvaticus',
 'Agaricus sylvicola',
 'Agaricus urinascens',
 'Agaricus xanthodermus',
 'Agrocybe praecox']

In [13]:
searcher_downloader(lista_setas_2)



[!!]Indexing page: 1

[%] Indexed 20 Images on Page 1.


[%] Downloading Image #1 from https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/Abortiporus_biennis_197275.jpg/1200px-Abortiporus_biennis_197275.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Abortiporus_biennis_161023wa.jpg/1200px-Abortiporus_biennis_161023wa.jpg
[Error]Invalid image, not saving https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Abortiporus_biennis_161023wa.jpg/1200px-Abortiporus_biennis_161023wa.jpg

[!] Issue getting: https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Abortiporus_biennis_161023wa.jpg/1200px-Abortiporus_biennis_161023wa.jpg
[!] Error:: No active exception to reraise
[%] Downloading Image #2 from https://upload.wikimedia.org/wikipedia/commons/6/6e/Abortiporus_biennis_179245.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://warehouse1.indicia.org.uk/upload/p16f1d5bv21msg1g93jl512se53i2.jpg
[%

In [14]:
contenido = os.listdir('dataset')
contenido=sorted(contenido)
contenido

['Abortiporus biennis',
 'Adelphella babingtonii',
 'Agaricus arvensis',
 'Agaricus campestris',
 'Agaricus litoralis',
 'Agaricus sylvaticus',
 'Agaricus sylvicola',
 'Agaricus urinascens',
 'Agaricus xanthodermus',
 'Agrocybe praecox']

In [15]:
comprobar_longitudes_carpetas(contenido)

Abortiporus biennis 20
Adelphella babingtonii 21
Agaricus arvensis 24
Agaricus campestris 21
Agaricus litoralis 22
Agaricus sylvaticus 23
Agaricus sylvicola 21
Agaricus urinascens 21
Agaricus xanthodermus 20
Agrocybe praecox 21


In [16]:
standarize_photos(contenido)

renombrando...



homogeneizando tamaño fotos...



estandarizando pixels...


In [17]:
train_images=np.array(lista)
train_images.shape

(200, 200, 200, 3)

In [18]:
lista_categories=[]
for folder in tqdm(contenido):
    for i in range(20):
        lista_categories.append(folder)
        
train_labels=np.array(lista_categories)

In [19]:
train_labels

array(['Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Abortiporus biennis', 'Abortiporus biennis',
       'Adelphella babingtonii', 'Adelphella babingtonii',
       'Adelphella babingtonii', 'Adelphella babingtonii',
       'Adelphella babingtonii', 'Adelphella babingtonii',
       'Adelphella babingtonii', 'Adelphella babingtonii',
       'Adelphella babingtonii', 'Adelphella babingtonii',
       'Adelphella babingtonii', 'Adelphella babingtonii',
       'Adelphella babingtonii', 'Adelphella babingtonii',
       'Adelphella babingtonii', 'Adelphella babingtonii'

In [20]:
test_labels=[]
for element in range(0,len(train_labels),20):
  test_labels.append(train_labels[element])
  test_labels.append(train_labels[element+1])

In [21]:
test_images=[]
for element in range(0,len(train_images),20):
  test_images.append(train_images[element])
  test_images.append(train_images[element+1])

In [22]:
len(train_images)

200

In [24]:
train_images_list=list(train_images)
for element in range(0,len(train_images),20):
  train_images_list.remove(train_images[element])
  train_images_list.remove(train_images[element+1])

ValueError: ignored

In [ ]:
train_labels_list=list(train_labels)
for element in range(0,len(train_labels),20):
  train_labels_list.remove(train_labels[element])
  train_labels_list.remove(train_labels[element+1])

In [ ]:
hasta aqui

In [ ]:
plot_mushroom_image(39,train_images,train_labels)

In [ ]:
train_labels.shape
#cambiar formato nombres para que funcionen

In [ ]:
type(np.unique(train_labels))

In [ ]:
l1 = list(range(10))
l2 = list(np.unique(train_labels))
d = dict(zip(l1,l2))
#create a dictionary to use one hot encoder

In [ ]:
d

In [ ]:
train_labels_nums = sorted(list(d.keys())*20)

In [ ]:
len(train_labels_nums)

Transform to one hot encoder the 10 classes we hace for now

In [ ]:
#transformamos a onehot nuestros targets 
onehot_train=to_categorical(train_labels_nums,num_classes=10)
onehot_train.shape

Vamos a crear nuestra primera red neuronal

In [ ]:
def build():
  m = models.Sequential()

#le decimos el input para que el output 

  l0 = layers.Flatten(input_shape=(200,200,3))
  m.add(l0)

  l1 = layers.Dense(units=50, activation = 'relu')
  m.add(l1)

  l2 = layers.Dense(units=25, activation = 'relu')
  m.add(l2)

  l3 = layers.Dense(units=15, activation = 'relu')
  m.add(l3)

  l4 = layers.Dense(units=10, activation = 'softmax')
  m.add(l4)

  
#ponemos none, ya no hace falta relu al ser una , decir que por defecto ya seria none.

#numero random el 3 por ejemoplo que es el numero de neuronas a la salida

  m.summary()

  return m

In [ ]:
m=build()

In [ ]:
m.compile(optimizer=optimizers.Adam(),loss=losses.categorical_crossentropy,metrics=[metrics.categorical_accuracy])

In [ ]:
train_images.shape

In [ ]:
onehot_train.shape

In [ ]:
 h = m.fit (train_images,onehot_train,batch_size= 5,epochs=30,validation_split=0.1)